In [1]:
!python -V

Python 3.10.4


In [3]:
import pandas as pd

import pickle

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [4]:
import mlflow

TRACKING_URI = "sqlite:///mlflow.db"
EXPERIMENT_NAME = "nyc-taxi-experiment"

mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [5]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [6]:
df_train = read_dataframe('../data/green_tripdata_2022-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2022-02.parquet')

In [7]:
len(df_train), len(df_val)

(59603, 66097)

In [8]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [9]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [10]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [11]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

6.928931233573194

In [12]:
with open('../models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

### track experiments

In [14]:
with mlflow.start_run():

    mlflow.set_tag("test-runs", "vishal")

    mlflow.log_param("train-data-path", "../data/green_tripdata_2022-01.parquet")
    mlflow.log_param("valid-data-path", "../data/green_tripdata_2022-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    #mlflow.log_artifact(local_path="../models/lin_reg.bin", artifact_path="models_pickle")

In [15]:
import xgboost as xgb

In [16]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [19]:
train_df = xgb.DMatrix(X_train, label=y_train)
valid_df = xgb.DMatrix(X_val, label=y_val)

In [22]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train_df,
            num_boost_round=1000,
            evals=[(valid_df, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid_df)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [23]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 15, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[01:21:25] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:13.31140                           
[1]	validation-rmse:10.57479                           
[2]	validation-rmse:8.81973                            
[3]	validation-rmse:7.73708                            
[4]	validation-rmse:7.08787                            
[5]	validation-rmse:6.70563                            
[6]	validation-rmse:6.48173                            
[7]	validation-rmse:6.34728                            
[8]	validation-rmse:6.26329                            
[9]	validation-rmse:6.20779                            
[10]	validation-rmse:6.17490                           
[11]	validation-rmse:6.15143                           
[12]	validation-rmse:6.13486                           
[13]	validation-rmse:6.12549                           
[14]	validation-rmse:6.11566                           
[15]	validation-rmse:6.10908             

[144]	validation-rmse:5.99409                          
[145]	validation-rmse:5.99390                          
[146]	validation-rmse:5.99378                          
[147]	validation-rmse:5.99350                          
[148]	validation-rmse:5.99344                          
[149]	validation-rmse:5.99352                          
[150]	validation-rmse:5.99321                          
[151]	validation-rmse:5.99294                          
[152]	validation-rmse:5.99209                          
[153]	validation-rmse:5.99197                          
[154]	validation-rmse:5.99194                          
[155]	validation-rmse:5.99127                          
[156]	validation-rmse:5.99093                          
[157]	validation-rmse:5.99089                          
[158]	validation-rmse:5.99091                          
[159]	validation-rmse:5.99061                          
[160]	validation-rmse:5.99065                          
[161]	validation-rmse:5.99015                   

[290]	validation-rmse:5.97135                          
[291]	validation-rmse:5.97115                          
[292]	validation-rmse:5.97103                          
[293]	validation-rmse:5.97234                          
[294]	validation-rmse:5.97232                          
[295]	validation-rmse:5.97233                          
[296]	validation-rmse:5.97222                          
[297]	validation-rmse:5.97219                          
[298]	validation-rmse:5.97216                          
[299]	validation-rmse:5.97228                          
[300]	validation-rmse:5.97225                          
[301]	validation-rmse:5.97234                          
[302]	validation-rmse:5.97267                          
[303]	validation-rmse:5.97260                          
[304]	validation-rmse:5.97287                          
[305]	validation-rmse:5.97276                          
[306]	validation-rmse:5.97277                          
[307]	validation-rmse:5.97263                   

[436]	validation-rmse:5.96256                          
[437]	validation-rmse:5.96251                          
[438]	validation-rmse:5.96240                          
[439]	validation-rmse:5.96240                          
[440]	validation-rmse:5.96213                          
[441]	validation-rmse:5.96184                          
[442]	validation-rmse:5.96159                          
[443]	validation-rmse:5.96173                          
[444]	validation-rmse:5.96164                          
[445]	validation-rmse:5.96169                          
[446]	validation-rmse:5.96157                          
[447]	validation-rmse:5.96195                          
[448]	validation-rmse:5.96197                          
[449]	validation-rmse:5.96203                          
[450]	validation-rmse:5.96215                          
[451]	validation-rmse:5.96191                          
[452]	validation-rmse:5.96198                          
[453]	validation-rmse:5.96167                   

[31]	validation-rmse:5.98177                           
[32]	validation-rmse:5.98048                           
[33]	validation-rmse:5.97899                           
[34]	validation-rmse:5.97635                           
[35]	validation-rmse:5.97546                           
[36]	validation-rmse:5.97227                           
[37]	validation-rmse:5.96953                           
[38]	validation-rmse:5.96598                           
[39]	validation-rmse:5.96368                           
[40]	validation-rmse:5.96253                           
[41]	validation-rmse:5.96113                           
[42]	validation-rmse:5.95788                           
[43]	validation-rmse:5.95669                           
[44]	validation-rmse:5.95324                           
[45]	validation-rmse:5.95307                           
[46]	validation-rmse:5.94863                           
[47]	validation-rmse:5.94770                           
[48]	validation-rmse:5.94501                    

...

[573]	validation-rmse:5.91642                          
[574]	validation-rmse:5.91638                          
[575]	validation-rmse:5.91643                          
[576]	validation-rmse:5.91625                          
[577]	validation-rmse:5.91621                          
[578]	validation-rmse:5.91627                          
[579]	validation-rmse:5.91599                          
[580]	validation-rmse:5.91592                          
[581]	validation-rmse:5.91573                          
[582]	validation-rmse:5.91561                          
[583]	validation-rmse:5.91550                          
[584]	validation-rmse:5.91553                          
[585]	validation-rmse:5.91540                          
[586]	validation-rmse:5.91523                          
[587]	validation-rmse:5.91504                          
[588]	validation-rmse:5.91512                          
[589]	validation-rmse:5.91500                          
[590]	validation-rmse:5.91498                   

[719]	validation-rmse:5.89795                          
[720]	validation-rmse:5.89783                          
[721]	validation-rmse:5.89749                          
[722]	validation-rmse:5.89753                          
[723]	validation-rmse:5.89750                          
[724]	validation-rmse:5.89746                          
[725]	validation-rmse:5.89738                          
[726]	validation-rmse:5.89695                          
[727]	validation-rmse:5.89693                          
[728]	validation-rmse:5.89686                          
[729]	validation-rmse:5.89690                          
[730]	validation-rmse:5.89685                          
[731]	validation-rmse:5.89662                          
[732]	validation-rmse:5.89667                          
[733]	validation-rmse:5.89656                          
[734]	validation-rmse:5.89644                          
[735]	validation-rmse:5.89637                          
[736]	validation-rmse:5.89618                   

[865]	validation-rmse:5.88406                          
[866]	validation-rmse:5.88400                          
[867]	validation-rmse:5.88399                          
[868]	validation-rmse:5.88392                          
[869]	validation-rmse:5.88366                          
[870]	validation-rmse:5.88368                          
[871]	validation-rmse:5.88373                          
[872]	validation-rmse:5.88358                          
[873]	validation-rmse:5.88351                          
[874]	validation-rmse:5.88337                          
[875]	validation-rmse:5.88327                          
[876]	validation-rmse:5.88328                          
[877]	validation-rmse:5.88304                          
[878]	validation-rmse:5.88291                          
[879]	validation-rmse:5.88287                          
[880]	validation-rmse:5.88278                          
[881]	validation-rmse:5.88258                          
[882]	validation-rmse:5.88254                   

[9]	validation-rmse:6.80279                            
[10]	validation-rmse:6.61668                           
[11]	validation-rmse:6.47737                           
[12]	validation-rmse:6.37294                           
[13]	validation-rmse:6.29263                           
[14]	validation-rmse:6.23331                           
[15]	validation-rmse:6.18474                           
[16]	validation-rmse:6.14425                           
[17]	validation-rmse:6.11645                           
[18]	validation-rmse:6.09213                           
[19]	validation-rmse:6.07390                           
[20]	validation-rmse:6.05747                           
[21]	validation-rmse:6.04568                           
[22]	validation-rmse:6.03593                           
[23]	validation-rmse:6.02735                           
[24]	validation-rmse:6.02148                           
[25]	validation-rmse:6.01707                           
[26]	validation-rmse:6.01301                    

KeyboardInterrupt: 

In [18]:
mlflow.xgboost.autolog(disable=True)

In [27]:
with mlflow.start_run():
    
    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train_df,
        num_boost_round=100,
        evals=[(valid_df, 'validation')],
        early_stopping_rounds=10
    )

    y_pred = booster.predict(valid_df)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("../models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("../models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[01:39:15] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:16.05222
[1]	validation-rmse:14.80991
[2]	validation-rmse:13.70427
[3]	validation-rmse:12.72336
[4]	validation-rmse:11.85427
[5]	validation-rmse:11.08754
[6]	validation-rmse:10.40971
[7]	validation-rmse:9.81638
[8]	validation-rmse:9.29388
[9]	validation-rmse:8.83723
[10]	validation-rmse:8.44318
[11]	validation-rmse:8.09925
[12]	validation-rmse:7.80065
[13]	validation-rmse:7.54272
[14]	validation-rmse:7.32061
[15]	validation-rmse:7.13082
[16]	validation-rmse:6.96400
[17]	validation-rmse:6.82314
[18]	validation-rmse:6.70494
[19]	validation-rmse:6.59879
[20]	validation-rmse:6.50899
[21]	validation-rmse:6.43078
[22]	validation-rmse:6.36401
[23]	validation-rmse:6.30726
[24]	validation-rmse:6.25598
[25]	validation-rmse:6.21334
[26]	validation-rmse:6.17380
[27]	validation-rmse:6.14228
[28]	validation-rmse:6.11374
[29]	validation-rmse:6.08791
[30]	validatio

In [23]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

/Users/cristian.martinez/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
